# Senator Filings Analysis

***

## Imports

In [1]:
from collections import defaultdict
import datetime as dt
from functools import lru_cache
import json
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf

## Introduction

In this notebook, we explore stock orders that were publicly filed by U.S. senators. The filings are scraped from https://efdsearch.senate.gov/search/. We calculate the returns of each senator by mimicking their buys and sells.

***

## Loading data

The `senators.pickle` file is scraped using the script in the root of the repository.

In [2]:
with open('senators.pickle', 'rb') as f:
    raw_senators_tx = pickle.load(f)

## Data cleaning

### Filling in missing tickers

In this section, we fill in as many of the missing ticker symbols as we can.

In [3]:
def tokenize(asset_name):
    """ Convert an asset name into useful tokens. """
    token_string = asset_name\
        .replace('(', '')\
        .replace(')', '')\
        .replace('-', ' ')\
        .replace('.', '')
    return token_string.split(' ')

def token_is_ticker(token, token_blacklist):
    return len(token) <= 4 and token.upper() not in token_blacklist

# These generic words do not help us determine the ticker
with open('blacklist.json', 'r') as f:
    blacklist = set(json.load(f))

missing_tickers = set(raw_senators_tx[
    (raw_senators_tx['ticker'] == '--')
    | (raw_senators_tx['ticker'] == '')
]['asset_name'])

ticker_map = {}
unmapped_tickers = set()
for m in missing_tickers:
    tokens = tokenize(m)
    if token_is_ticker(tokens[0], blacklist):
        ticker_map[m] = tokens[0].upper()
    elif token_is_ticker(tokens[-1], blacklist):
        ticker_map[m] = tokens[-1].upper()
    else:
        unmapped_tickers.add(m)

As a second pass, we assign tickers to asset names that have any of the specified keywords.

In [4]:
phrase_to_ticker = {
    'FOX': 'FOX',
    'AMAZON': 'AMZN',
    'AARON': 'AAN',
    'ALTRIA': 'MO',
    'APPLE': 'AAPL',
    'CHEVRON': 'CVX',
    'DUPONT': 'DD',
    'ALPHABET': 'GOOGL',
    'GOOG': 'GOOGL',
    'GENERAL ELECTRIC': 'GE',
    'JOHNSON': 'JNJ',
    'NEWELL': 'NWL',
    'OWENS': 'OMI',
    'PFIZER': 'PFE',
    'TYSON': 'TSN',
    'UNDER ARMOUR': 'UAA',
    'VERIZON': 'VZ',
    'WALT': 'DIS'
}

for m in unmapped_tickers:
    for t in phrase_to_ticker:
        if t in m.upper():
            ticker_map[m] = phrase_to_ticker[t]

tx_with_tickers = raw_senators_tx.copy()
for a, t in ticker_map.items():
    tx_with_tickers.loc[tx_with_tickers['asset_name'] == a, 'ticker'] = t

### Filtering rows and columns

We filter out useless rows and missing symbols, and then add some useful columns for the final dataset.

In [5]:
filtered_tx = tx_with_tickers[tx_with_tickers['ticker'] != '--']
filtered_tx = filtered_tx.assign(
    ticker=filtered_tx['ticker'].map(
        lambda s: s.replace('--', '').replace('\n', '')))

filtered_tx = filtered_tx[filtered_tx['order_type'] != 'Exchange']

In [6]:
def parse_tx_amount(amt):
    """ Get the lower bound for the transaction amount. """
    return int(amt.replace('Over $50,000,000', '50000000')
               .split(' - ')[0]
               .replace(',', '')
               .replace('$', ''))

senators_tx = filtered_tx.assign(
    tx_estimate=filtered_tx['tx_amount'].map(parse_tx_amount))
senators_tx = senators_tx.assign(
    full_name=senators_tx['first_name']
        .str
        .cat(senators_tx['last_name'], sep=' ')
)
useful_cols = [
    'file_date',
    'tx_date',
    'full_name',
    'order_type',
    'ticker',
    'tx_estimate'
]
senators_tx = senators_tx[useful_cols]
senators_tx = senators_tx.assign(
    tx_date=senators_tx['tx_date'].map(
        lambda v: dt.datetime.strptime(v.strip(), '%m/%d/%Y')))
senators_tx = senators_tx.assign(
    file_date=senators_tx['file_date'].map(
        lambda v: dt.datetime.strptime(v.strip(), '%m/%d/%Y')))
senators_tx

,file_date,tx_date,full_name,order_type,ticker,tx_estimate
0,2024-04-24,2024-03-22,Sheldon Whitehouse,Sale (Full),UL,1001
1,2024-04-24,2024-03-13,Sheldon Whitehouse,Sale (Full),CMCSA,15001
2,2024-04-24,2024-04-15,Rafael E Cruz,Sale (Partial),GS,250001
3,2024-04-19,2024-04-04,Tina Smith,Sale (Partial),PODD,50001
4,2024-04-19,2024-04-04,Tina Smith,Sale (Partial),NEE,100001
...,...,...,...,...,...,...
16457,2014-01-31,2014-01-29,Susan M Collins,Sale (Partial),C,1001
16458,2014-01-31,2014-01-28,Susan M Collins,Sale (Partial),EBAY,1001
16459,2014-01-31,2014-01-24,Susan M Collins,Purchase,FB,1001
16460,2014-01-31,2014-01-24,Susan M Collins,Purchase,CRM,1001


## Returns calculation

These cells help us download the market data for the specified tickers. We store the market data in files so we don't need to repeatedly download the same information.

In [7]:
def download_for_ticker(ticker, check_cache=True):
    """ Download a file of stock prices for this ticker to disk. """
    if check_cache and os.path.exists('stocks/{0}.pickle'.format(ticker)):
        return
    d = yf.Ticker(ticker)
    with open('stocks/{0}.pickle'.format(ticker), 'wb') as f:
        pickle.dump({
            'price': d.history(period='max').reset_index()
        }, f)

def load_for_ticker(ticker):
    """ Load the file of stock prices for this ticker. """
    with open('stocks/{0}.pickle'.format(ticker), 'rb') as f:
        dump = pickle.load(f)
    raw = dump['price']
    return raw[['Date', 'Close']]\
        .rename(columns={'Date': 'date', 'Close': 'price'})

def _price_for_date(df, date):
    """ Helper function for `ticker_at_date`. """
    df = df[df['date'] >= date].sort_values(by='date')
    return df['price'].iloc[0]

@lru_cache(maxsize=128)
def ticker_at_date(ticker, date):
    """
    Price of a ticker at a given date. Raise an IndexError if there is no
    such price.
    """
    try:
        data = load_for_ticker(ticker)
        # Sell at the next opportunity possible
        return _price_for_date(data, date)
    except Exception:
        # If any exception occurs, refresh the cache
        download_for_ticker(ticker, check_cache=False)
        data = load_for_ticker(ticker)
        return _price_for_date(data, date)

In [8]:
try:
    os.mkdir('stocks')
except FileExistsError:
    pass

all_tickers = set(senators_tx['ticker'])
for i, t in enumerate(all_tickers):
    if i % 100 == 0:
        print('Working on ticker {0}'.format(i))
    try:
        download_for_ticker(t)
    except Exception as e:
        print('Ticker {0} failed with exception: {1}'.format(t, e))

Working on ticker 0


OAK: No timezone found, symbol may be delisted
CELP: No timezone found, symbol may be delisted
XL: No timezone found, symbol may be delisted
ZNGA: No timezone found, symbol may be delisted
KRFT: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
Q: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
AGU: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
HUB-B: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
FEYE: No timezone found, symbol may be delisted
MDCA: No timezone found, symbol may be delisted
DISCA: No timezone found, symbol may be delisted
DGNR: No timezone found, symbol may be delisted
ECOM: No timezone found, symbol may be delisted
CLNY: No timezone found, symbol may be delisted
ITR: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
LDR: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
CONE: No timezone found, symbol may be delisted

Working on ticker 100


MCC: No timezone found, symbol may be delisted
KSU: No timezone found, symbol may be delisted
BUNT-RP: No timezone found, symbol may be delisted
JCOM: No timezone found, symbol may be delisted
HCN: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
ANTM: No timezone found, symbol may be delisted
CNR: No timezone found, symbol may be delisted
BEAV: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
BBT: No timezone found, symbol may be delisted
TCP: No timezone found, symbol may be delisted
WFT: No timezone found, symbol may be delisted
KMP: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
IACI: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
OAS: No timezone found, symbol may be delisted
FDC: No timezone found, symbol may be delisted


Working on ticker 200


...

AGN: No timezone found, symbol may be delisted
LINE: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
CMN: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
SQI: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
ATH: No timezone found, symbol may be delisted
VTTI: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
EXXI: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
MTSC: No timezone found, symbol may be delisted
EQM: No timezone found, symbol may be delisted
LMACU: No timezone found, symbol may be delisted
TWC: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
SJR: No timezone found, symbol may be delisted
CTRL: No timezone found, symbol may be delisted
LGP: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
MYL: No timezone found, symbol may be delisted
NPSND: No price data found, symbol may be delisted (1d 1925-05-2

Working on ticker 1400


DWDP: No timezone found, symbol may be delisted
CMLP: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
CTECBX: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
WTR: No timezone found, symbol may be delisted
BKEP: No timezone found, symbol may be delisted
CJES: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
ITYBY: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
VPFG: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
COF-PP: No timezone found, symbol may be delisted
LSI: No timezone found, symbol may be delisted
LTXB: No timezone found, symbol may be delisted
UMPQ: No timezone found, symbol may be delisted


Working on ticker 1500


TYC: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
RDS-B: No timezone found, symbol may be delisted
SPY160219P00180000: No timezone found, symbol may be delisted
EVOP: No timezone found, symbol may be delisted
TUMI: No price data found, symbol may be delisted (1d 1925-05-21 -> 2024-04-27)
CY: No timezone found, symbol may be delisted


### Mimicking buy + sell orders

We calculate a given senator's return by calculating the return between each buy or sell order, and then solving for the cumulative return. We convert that to a CAGR given the time period the senator was investing.

We keep track of how many units of each stock a senator is holding. If we ever see a filing that indicates the senator sold more than we estimated they are holding, we just sell all of the units we have on record. (We do not allow the senator to go short.)

In [9]:
buckets = [
    (1000, 15000),
    (15000, 50000),
    (50000, 100000),
    (100000, 250000),
    (250000, 500000),
    (500000, 1000000),
    (1000000, 5000000),
    (5000000, 25000000),
    (25000000, 50000000),
    (50000000, float('inf'))
]

def same_bucket(dollar_value_a, dollar_value_b):
    """
    If the dollar value of the stock units is roughly the same, sell all
    units.
    """
    for v1, v2 in buckets:
        if dollar_value_a >= v1 and dollar_value_a < v2:
            return dollar_value_b >= v1 and dollar_value_b < v2
    return False

def portfolio_value(stocks, date):
    """
    Value of a portfolio if each ticker has the specified number of units.
    """
    v = 0
    for s, units in stocks.items():
        if units == 0:
            continue
        try:
            v += ticker_at_date(s, date) * units
        except IndexError as e:
            # Swallow missing ticker data exception
            pass
    return v

def calculate_return(before_values,
                     after_values,
                     begin_date,
                     end_date,
                     tx_dates):
    """
    Calculate cumulative return and CAGR given the senators portfolio
    value over time.
    """
    before_values.pop(0)
    after_values.pop(-1)
    # We calculate the total return by calculating the return
    # between each transaction, and solving for the cumulative
    # return.
    growth = np.array(before_values) / np.array(after_values)
    portfolio_return = np.prod(growth[~np.isnan(growth)])
    years = (end_date - begin_date).days / 365
    if years == 0:
        cagr = 0
    else:
        cagr = portfolio_return**(1 / years)
    # DataFrame of cumulative return
    tx_dates.pop(0)
    tx_dates = np.array(tx_dates)
    tx_dates = tx_dates[~np.isnan(growth)]
    cumulative_growth = np.cumprod(growth[~np.isnan(growth)])
    growth_df = pd.DataFrame({
        'date': tx_dates,
        'cumulative_growth': cumulative_growth
    })
    return {
        'portfolio_return': portfolio_return,
        'annual_cagr': cagr,
        'growth': growth_df
    }

def return_for_senator(rows, date_col='tx_date'):
    """
    Simulate a senator's buy and sell orders, and calculate the
    return.
    """
    stocks = defaultdict(int)
    # Value of portfolio at various timepoints to calculate return
    portfolio_value_before_tx = []
    portfolio_value_after_tx = []
    tx_dates = []
    rows = rows.sort_values(by=date_col)
    for _, row in rows.iterrows():
        date = row[date_col]
        if date_col == 'file_date':
            # We can't execute the trade the same day
            date += dt.timedelta(days=1)
        try:
            stock_price = ticker_at_date(row['ticker'], date)
        except IndexError as e:
            # Skip the row if we're missing ticker data
            continue
        value_before_tx = portfolio_value(stocks, date)
        if 'Purchase' in row['order_type']:
            tx_amt = row['tx_estimate']
            n_units = tx_amt / ticker_at_date(row['ticker'], date)
            stocks[row['ticker']] += n_units
        elif 'Sale' in row['order_type']:
            current_value = stock_price * stocks[row['ticker']]
            if 'Full' in row['order_type'] or\
                same_bucket(row['tx_estimate'], current_value):
                stocks[row['ticker']] = 0
            else:
                new_n_units = stocks[row['ticker']] - \
                    row['tx_estimate'] / stock_price
                stocks[row['ticker']] = max(0, new_n_units)
        portfolio_value_before_tx.append(value_before_tx)
        portfolio_value_after_tx.append(portfolio_value(stocks, date))
        tx_dates.append(date)
    return calculate_return(
        portfolio_value_before_tx,
        portfolio_value_after_tx,
        begin_date=min(rows[date_col]),
        end_date=max(rows[date_col]),
        tx_dates=tx_dates
    )

In [10]:
senator_returns = []
senator_tx_growth = {}
senator_file_growth = {}
senator_names = set(senators_tx['full_name'])

The following cell took my laptop about three hours to run.

In [11]:
failed_senators = {}
print('{} senators total'.format(len(senator_names)))
for n in senator_names:
    print('Starting {}'.format(n))
    if n in senator_tx_growth:
        # Don't re-calculate for a given senator
        continue
    try:
        tx_return = return_for_senator(
            senators_tx[senators_tx['full_name'] == n],
            date_col='tx_date')
        file_return = return_for_senator(
            senators_tx[senators_tx['full_name'] == n],
            date_col='file_date')
        senator_returns.append({
            'full_name': n,
            'tx_total_return': tx_return['portfolio_return'],
            'tx_cagr': tx_return['annual_cagr'],
            'file_total_return': file_return['portfolio_return'],
            'file_cagr': file_return['annual_cagr']
        })
        senator_tx_growth[n] = tx_return['growth']
        senator_file_growth[n] = file_return['growth']
    except Exception as e:
        print('Failed senator {0} with exception {1}'.format(n, e))
        failed_senators[n] = e

62 senators total
Starting Maria Cantwell


RNWK: No timezone found, symbol may be delisted
RNWK: No timezone found, symbol may be delisted
RNWK: No timezone found, symbol may be delisted
RNWK: No timezone found, symbol may be delisted
RNWK: No timezone found, symbol may be delisted
RNWK: No timezone found, symbol may be delisted


Failed senator Maria Cantwell with exception pop from empty list
Starting Markwayne Mullin
Failed senator Markwayne Mullin with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting Jon Kyl


TWTR: No timezone found, symbol may be delisted
TWTR: No timezone found, symbol may be delisted


Failed senator Jon Kyl with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting Michael  B Enzi
Failed senator Michael  B Enzi with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting James M Inhofe
Failed senator James M Inhofe with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting John Hoeven
Failed senator John Hoeven with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting John P Ricketts
Failed senator John P Ricketts with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting Gary C Peters
Failed senator Gary C Peters with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting Ron L Wyden


...

Failed senator Ladda Tammy Duckworth with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting Cynthia M Lummis
Failed senator Cynthia M Lummis with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting William F Hagerty, IV
Failed senator William F Hagerty, IV with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting John N Kennedy
Failed senator John N Kennedy with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting Robert P Casey, Jr.
Failed senator Robert P Casey, Jr. with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting Joseph Manchin, III
Failed senator Joseph Manchin, III with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting Susan M Collins
Failed senator Susan M Collins with exception Invalid compar

APC: No timezone found, symbol may be delisted


Failed senator Jacklyn S Rosen with exception pop from empty list
Starting Rick Scott


WTT: No timezone found, symbol may be delisted
WTT: No timezone found, symbol may be delisted
WTT: No timezone found, symbol may be delisted


Failed senator Rick Scott with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting John Cornyn
Failed senator John Cornyn with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting Kelly Loeffler
Failed senator Kelly Loeffler with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Starting Rafael E Cruz
Failed senator Rafael E Cruz with exception Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp


We look at the results to see the senators that outperformed the market.

In [12]:
def plot_senator_growth(growth):
    """ Plot the senator's portfolio growth against the S&P 500. """
    plt.plot_date(growth['date'], growth['cumulative_growth'], '-')
    download_for_ticker('SPY')
    spy = load_for_ticker('SPY')
    spy = spy[(spy['date'] >= min(growth['date']))
              & (spy['date'] <= max(growth['date']))]
    spy_prices = spy['price']
    spy_growth = np.cumprod(np.diff(spy_prices) / spy_prices[1:] + 1)
    dates = spy['date'].iloc[1:]
    plt.plot_date(dates, spy_growth, '-')
    plt.show()
    print('Earliest date: {}'.format(min(growth['date'])))
    print('Latest date: {}'.format(max(growth['date'])))
    print('Market return: {}'.format(
        spy_prices.iloc[-1] / spy_prices.iloc[0]))
    senator_growth = growth['cumulative_growth']
    print('Senator return: {}'.format(
        senator_growth.iloc[-1] / senator_growth.iloc[0]))

In [13]:
returns = pd.DataFrame(senator_returns)
returns = returns[(returns['tx_total_return'] > returns['tx_cagr'])
                  & (returns['tx_cagr'] > 0)]
returns.sort_values('tx_cagr')

KeyError: 'tx_total_return'

In [ ]:
plot_senator_growth(senator_tx_growth['Angus S King, Jr.'])

## About this notebook

Author: Neel Somani, Software Engineer

Email: neel@berkeley.edu

Website: https://www.ocf.berkeley.edu/~neel/

Updated On: 2020-05-23